In [3]:
import torch

x = torch.tensor([[5.,4.,2.5],[2.,3.,2.1]])
print(x.size())
print(x.dtype)
x

torch.Size([2, 3])
torch.float32


tensor([[5.0000, 4.0000, 2.5000],
        [2.0000, 3.0000, 2.1000]])

In [ ]:
w = torch.tensor([1.,2.,3.], requires_grad=True)

b = torch.tensor([0.], requires_grad=True)

y = torch.tensor([10.,9.])

y_cal = torch.matmul(x,w) + b

print(y_cal)

tensor([20.5000, 14.3000], grad_fn=<AddBackward0>)


In [7]:
#cost function
J = ((y_cal - y)**2).sum()

#calculate gradient
J.backward()

print(w.grad, b.grad)

tensor([126.2000, 115.8000,  74.7600]) tensor([31.6000])


## **Implement** MLP in pyTorch
   #### To write feed forward neural network using py-torch with one **input layer** , one **hidden layer** and one **output layer**

   ## 1.1 Load Data

In [6]:
#load library
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

#Load data from minst file using np

minst_ = np.load('mnist_scaled.npz')

X_train, y_train, X_test, y_test = [minst_[f] for f in ['X_train', 'y_train', 'X_test', 
                                                        'y_test']]

X_train = X_train.reshape(-1,1,28,28)
X_test = X_test.reshape(-1,1,28,28)

del minst_

print(f'X_train shape {X_train.shape}')
print(f'y_train shape {y_train.shape}')
print(f'X_test shape {X_test.shape}')
print(f'y_test shape {y_test.shape}')

#Preparing dataset numpy to tensor datatype for pytorch operations 

train_dataset = TensorDataset(torch.tensor(X_train[:55000], dtype=(torch.float32)),
                                torch.tensor(y_train[:55000], dtype=(torch.int64)))
test_dataset = TensorDataset(torch.tensor(X_train[55000:], dtype=(torch.float32)),
                                torch.tensor(y_train[55000 :], dtype=(torch.int64)))
val_dataset = TensorDataset(torch.tensor(X_test, dtype=(torch.float32)),
                                torch.tensor(y_test, dtype=(torch.int64)))

X_train shape (60000, 1, 28, 28)
y_train shape (60000,)
X_test shape (10000, 1, 28, 28)
y_test shape (10000,)


In [ ]:
#mini batch size
minibatch_size = 300

#Data loaders
train_dataloader = DataLoader(dataset=train_dataset, batch_size= minibatch_size, shuffle = True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size= minibatch_size, shuffle = False)
val_dataloader = DataLoader(dataset=val_dataset, batch_size= minibatch_size, shuffle = False)

### To define network class

In [20]:
class NeuralNet_pytorch(nn.Module):
    ''' 
    NeuralNet class for forward pass functionality

    Parameter:
    ==================================

    n_inputs: {int}
    number of inputs to the model

    n_hidden: {int}
    number of hidden units

    n_outputs: {int}
    number of output units

    Methods:
    =====================================

    forward(X): [input parameter - prepared(X_train, X_test) dataset]
    sequential forward calculation of nn

    '''
    def __init__(self, n_inputs, n_hidden=30, n_output=1):
        super(NeuralNet_pytorch, self).__init__()
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output

        self.model_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.n_inputs, self.n_hidden),
            nn.Sigmoid(),
            nn.Linear(self.n_hidden, self.n_output),
        )

    def forward(self,X):
        out = self.model_layer(X)
        return out

### Define test function to find the **accuracy** of validation dataset

In [16]:
def test(dataloader, model):

    #read dataloader size
    dataset_size = len(dataloader.dataset)

    #model should be evaluate mode
    model.eval()

    #No gradient calculations
    with torch.no_grad():

        correct=0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            #forward propagation
            output = model(images)

            #derive prediction
            y_pred = output.argmax(1)

            #getting the corrected prediction count
            correct += (y_pred == labels).sum().item()

        #calculate accuracy
        acc = correct/dataset_size

    return acc

## **Apply** Neural net for **Image recognition** 

### Define parameters

In [22]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device : {device}')

#parameters

n_input = 28*28
n_hidden =100
n_output = 10
learning_rate = 0.0005
epochs = 100
shuffle = True

Using device : cpu


In [21]:
# instanciate the model

model = NeuralNet_pytorch(n_input, n_hidden, n_output).to(device)
print(model)

NeuralNet_pytorch(
  (model_layer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=100, bias=True)
    (2): Sigmoid()
    (3): Linear(in_features=100, out_features=10, bias=True)
  )
)


### fit() model

In [26]:
#instance creation for loss and optimizer
loss = nn.CrossEntropyLoss()
optimize = torch.optim.Adam(model.parameters(), lr=learning_rate)

#train over epochs
for epoch in range(epochs):
    correct=0; sample=0

    for idx, (images,labels) in enumerate(train_dataloader):

        images, labels = images.to(device), labels.to(device)

        #forward pass
        output = model(images)

        #back propagation and gradient descent
        optimize.zero_grad()
        J = loss(output, labels)
        J.backward()
        #adjust the parameters
        optimize.step()

        _, y_pred = torch.max(output.data, 1)

        #calculate the output matching the label
        correct += (y_pred == labels).sum().item()
        sample += labels.size(0)

    if (epoch + 1) % 5 == 0:

        train_acc = correct/sample
        test_acc = test(test_dataloader, model)

        # Print info
        print (f"Epoch [{epoch+1}/{epochs}], Loss: {J.item():.4f}")
        print(f"      Training accuracy: {train_acc*100:.1f}%")
        print(f"      Validation accuracy: {test_acc*100:.1f}%")


Epoch [5/100], Loss: 0.0090
      Training accuracy: 100.0%
      Validation accuracy: 97.9%
Epoch [10/100], Loss: 0.0082
      Training accuracy: 100.0%
      Validation accuracy: 97.9%
Epoch [15/100], Loss: 0.0050
      Training accuracy: 100.0%
      Validation accuracy: 97.8%
Epoch [20/100], Loss: 0.0073
      Training accuracy: 100.0%
      Validation accuracy: 97.9%
Epoch [25/100], Loss: 0.0025
      Training accuracy: 100.0%
      Validation accuracy: 98.0%
Epoch [30/100], Loss: 0.0025
      Training accuracy: 100.0%
      Validation accuracy: 97.9%
Epoch [35/100], Loss: 0.0022
      Training accuracy: 100.0%
      Validation accuracy: 97.9%
Epoch [40/100], Loss: 0.0005
      Training accuracy: 100.0%
      Validation accuracy: 98.0%
Epoch [45/100], Loss: 0.0009
      Training accuracy: 100.0%
      Validation accuracy: 98.0%
Epoch [50/100], Loss: 0.0013
      Training accuracy: 100.0%
      Validation accuracy: 97.9%
Epoch [55/100], Loss: 0.0006
      Training accuracy: 100.0%
